# Copy illumination correction files


Illumination organisation

```
images
├── 2023_03_01_Batch2_U2OS
│  ├── illum
│  │  ├── <plate-name>
|  │  │  ├── <plate-name>_Illum<Channel>.npy
|  │  │  ├── <plate-name>_Illum<Channel>.npy
|  │  │  ...
│  │  ├── <plate-name>
│  │  │  ...
│  └── images
├── <Batch_name>
├── ...
```

Channel names:
IllumDNA
IllumER
IllumAGP
IllumMito

In [2]:
import os
import sys
import glob
import shutil
import pandas as pd

# Specify the directory path
data_input_path = '/home/schmiedc/FMP_Docs/Projects/CellPainting/DataUpload/TestInput_corr/'

data_output_path = '/home/schmiedc/FMP_Docs/Projects/CellPainting/DataUpload/TestOutput/'

# Load same key_file for transformation
key_file = pd.read_csv(data_input_path + 'FMP_U2OS_keyfile.csv')  

# Path to illumination files
illum_path = "/home/schmiedc/FMP_Docs/Projects/CellPainting/DataUpload/IllumFiles/U2OS_10uM/"

cpg_name = None
source = None

# make sure that there is only one source in the key_file
cpg_name_number = len(key_file['cpg_name'].unique())
source_number = len(key_file['source'].unique())

if source_number == 1 & cpg_name_number == 1:
    
    print("Only 1 source and 1 cpg name present in key_file: proceed")
    cpg_name = key_file['cpg_name'][0]
    source = key_file['source'][0]
    
else:
    
    print("Source number and/or cpg name number incorrect")
    
# generate paths
cpg_path = os.path.join(data_output_path, cpg_name)
source_path = os.path.join(cpg_path, source)
images_path = os.path.join(source_path, 'images')

def copy_rename(input_path, input_file_name, destination_path, update_base_name, file_ending):
    
    print("Copy and rename: " + input_file_name )
    
    original_file_path = input_path + os.path.sep + input_file_name

    # copy from original path
    shutil.copy(original_file_path, destination_path)

    # Create new path
    new_file_path = destination_path + os.path.sep + update_base_name + file_ending

    # rename file with new name
    shutil.move(destination_path + os.path.sep + input_file_name, new_file_path)
    
    print("New path: " + new_file_path )


log = open(data_input_path + os.path.sep + 'log_copy_illum_corr.txt','w')

# For batch_name 
batch_name_list = key_file['Batch_Name'].unique()

for batch_name in batch_name_list:
    
    print('Batch Name: ' + batch_name)
    
    filtered_key_file = key_file[key_file['Batch_Name'] == batch_name]
    
    # Creates Batch folder using <Batch_Name> in <Source>/images/
    batch_path = os.path.join(images_path, batch_name)

    # create images folder in Batch folder
    batch_illum_path = os.path.join(batch_path, 'illum')
    
    try:
        os.mkdir(batch_illum_path)
    except OSError as error:  
        print(error, file=log)
    
    print("Created illum dir: " + batch_name)
    
    # For all plates per batch Assay_Plate_Barcode 
    assay_plate_barcode_list = filtered_key_file['Assay_Plate_Barcode'].unique()
    
    for assay_plate_barcode in assay_plate_barcode_list:
        
        print("Image folder name " + assay_plate_barcode)
        
        # Create plate name folder in illum folder
        barcode_filtered_key_file = filtered_key_file[filtered_key_file['Assay_Plate_Barcode'] == 
                                                      assay_plate_barcode]

        if barcode_filtered_key_file.shape[0] == 1:
    
            print(f"Plate name {assay_plate_barcode} is unique")
    
            # gets the values to filter the annotation file
            barcode_plate_name = barcode_filtered_key_file['plate_name'].iloc[0]
            barcode_replicate_number = barcode_filtered_key_file['replicate_number'].iloc[0]
            barcode_plate_Map_Name = barcode_filtered_key_file['Plate_Map_Name'].iloc[0]
            
            # create plate folder in Batch folder
            plate_illum_path = os.path.join(batch_illum_path, barcode_plate_Map_Name)
            
            try:
                os.mkdir(plate_illum_path)
            
            except OSError as error:
                print(error, file=log)
            
            org_plate_illum_path = os.path.join(illum_path, barcode_plate_Map_Name)
            
            # Check if original illum folder exists
            if os.path.exists(org_plate_illum_path):
                
                print(f"Orig. illum. folder for {barcode_plate_Map_Name} exists")
                illum_corr_file_list = glob.glob1(org_plate_illum_path,"*.npy")
                
                # count number of .npy illum corr files
                illum_corr_file_counter = len(illum_corr_file_list)
                
                # Check if these are 4
                if illum_corr_file_counter == 4:
                    
                    print(f"{barcode_plate_name} has all 4 illum. corr. files")
                    
                    # contains all permissable endings
                    illum_corr_endings = ['_IllumMito.npy', '_IllumDNA.npy', '_IllumER.npy', '_IllumAGP.npy']
                    
                    # dict to store the file names
                    illum_corr_file_names = {ending: None for ending in illum_corr_endings}
                    
                    # Iterate over the ill_corr_file_list and check for illum_corr_endings
                    for file in illum_corr_file_list:
                        for ending in illum_corr_endings:
                            if file.endswith(ending):
                                illum_corr_file_names[ending] = file
                                break
                                
                    # Check if all required files are found
                    missing_files = [ending for ending, file in illum_corr_file_names.items() if file is None]

                    if missing_files:
                        
                        print(f"Error: The following required files are missing: {', '.join(missing_files)}")
                        
                    else:
                        
                        for ending in illum_corr_endings:
                            
                            filename = illum_corr_file_names[ending]
                            copy_rename(org_plate_illum_path, filename, plate_illum_path, barcode_plate_Map_Name, ending)
                                        
                else:
                    
                    print(f"Error: incorrect number for illum. corr. files of {barcode_plate_Map_Name}", file=log)
                    
            else:
                print(f"Error: Orig. illum. folder for {barcode_plate_Map_Name} does not exists")
                
        else:
            
            print(f"Error: Plate name {assay_plate_barcode} not unique", file=log)

Only 1 source and 1 cpg name present in key_file: proceed
Batch Name: 2023_02_15_Batch1_U2OS
Created illum dir: 2023_02_15_Batch1_U2OS
Image folder name U2OSB1001R1__2023-02-15T15_21_49-Measurement-1
Plate name U2OSB1001R1__2023-02-15T15_21_49-Measurement-1 is unique
Orig. illum. folder for B1001_R1 exists
B1001 has all 4 illum. corr. files
Copy and rename: U2OS_10uM_B1001_R1_IllumMito.npy
New path: /home/schmiedc/FMP_Docs/Projects/CellPainting/DataUpload/TestOutput/cpgXXXX-EU-OS-bioactives/FMP/images/2023_02_15_Batch1_U2OS/illum/B1001_R1/B1001_R1_IllumMito.npy
Copy and rename: U2OS_10uM_B1001_R1_IllumDNA.npy
New path: /home/schmiedc/FMP_Docs/Projects/CellPainting/DataUpload/TestOutput/cpgXXXX-EU-OS-bioactives/FMP/images/2023_02_15_Batch1_U2OS/illum/B1001_R1/B1001_R1_IllumDNA.npy
Copy and rename: U2OS_10uM_B1001_R1_IllumER.npy
New path: /home/schmiedc/FMP_Docs/Projects/CellPainting/DataUpload/TestOutput/cpgXXXX-EU-OS-bioactives/FMP/images/2023_02_15_Batch1_U2OS/illum/B1001_R1/B1001_R1_